In [ ]:
# export
from fastai2.basics import *
from transformers import AutoTokenizer

from fastai2_utils.data.all import *
from fastai_transformers_utils.all import *

In [ ]:
# default_exp data.news_commentary

In [ ]:
# all_skip

In [ ]:
# ori_data_loc = './data/News_Commentary/news-commentary-v14.en-zh.tsv'
# tok_data_loc = './data/News_Commentary/tok-news-commentary-v14.en-zh.csv'
# enc_tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')
# dec_tokenizer = GPT2DecoderTokenizer.from_pretrained('distilgpt2')
# enc_seq_len = 65
# dec_seq_len = 100

In [ ]:
ori_data_loc = './data/News_Commentary/news-commentary-v14.en-zh.tsv'
tok_data_loc = './data/News_Commentary/tok-news-commentary-v14.en-zh.csv'
enc_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
dec_tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')
enc_seq_len = 100
dec_seq_len = 65

# Data News_Commentary
> English to Chinese pairs

## tokenize_data

In [ ]:
# # export
# def _tokenize_data(ori_data_loc, enc_tokenizer, dec_tokenizer):
#     df = pd.read_csv(ori_data_loc, header=None, names=['English', 'Chinese'], delimiter='\t', error_bad_lines=False, warn_bad_lines=False)
#     df.dropna(inplace=True)
#     df.reset_index(drop=True, inplace=True)
    
#     tok_np = np.zeros((len(df), 2)).astype('object')
#     for i, eng_str, chn_str in progress_bar(tuple(df.itertuples())):
#         tok_ch = ' '.join(enc_tokenizer.tokenize(chn_str))
#         tok_en = ' '.join(dec_tokenizer.tokenize(eng_str))
#         tok_np[i, 0] = tok_en
#         tok_np[i, 1] = tok_ch
#     tok_df = pd.DataFrame(tok_np, columns=['English', 'Chinese'])

#     return tok_df

In [ ]:
# export
def _tokenize_data(ori_data_loc, enc_tokenizer, dec_tokenizer):
    df = pd.read_csv(ori_data_loc, header=None, names=['English', 'Chinese'], delimiter='\t', error_bad_lines=False, warn_bad_lines=False)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    tok_np = np.zeros((len(df), 2)).astype('object')
    for i, eng_str, chn_str in progress_bar(tuple(df.itertuples())):
        tok_en = ' '.join(enc_tokenizer.tokenize(eng_str))
        tok_ch = ' '.join(dec_tokenizer.tokenize(chn_str))
        tok_np[i, 0] = tok_en
        tok_np[i, 1] = tok_ch
    tok_df = pd.DataFrame(tok_np, columns=['English', 'Chinese'])

    return tok_df

In [ ]:
# export
def _get_eda_tok_df(tok_df):
    eng_len = tok_df.English.apply(lambda x: len(x.split()))
    chn_len = tok_df.Chinese.apply(lambda x: len(x.split()))
    eda_tok_df = pd.DataFrame(data={'eng_len': eng_len, 'chn_len': chn_len}, index=tok_df.index)
    return eda_tok_df

In [ ]:
# export
def _clean_data(tok_df):
    eda_tok_df = _get_eda_tok_df(tok_df)
    tok_df.drop(index=tok_df[(eda_tok_df.chn_len > 200)].index, inplace=True)
    tok_df.reset_index(drop=True, inplace=True)
    return tok_df

In [ ]:
# export
def _small_data(tok_df):
    eda_tok_df = _get_eda_tok_df(tok_df)
    tok_df.drop(index=tok_df[(eda_tok_df.chn_len > 62)].index, inplace=True)
    tok_df.reset_index(drop=True, inplace=True)
    return tok_df

In [ ]:
# export
def _add_isvalid(tok_df):
    is_valid = np.zeros(len(tok_df))
    is_valid[:int(len(tok_df)*0.2)] = 1
    np.random.RandomState(42).shuffle(is_valid)
    is_valid = is_valid.astype(np.bool)
    tok_df['is_valid'] = is_valid
    return tok_df

In [ ]:
# skip
# tokenize, clean data and save it to tok_data_loc
tok_df = _tokenize_data(ori_data_loc, enc_tokenizer, dec_tokenizer)
tok_df = _clean_data(tok_df)
tok_df = _small_data(tok_df)
tok_df = _add_isvalid(tok_df)
tok_df.to_csv(tok_data_loc, index=False)
tok_df

,English,Chinese,is_valid
0,1929 or 1989 ?,1929 年 还 是 1989 年 ?,True
1,"PA ##RI ##S – As the economic crisis deep ##ens and widen ##s , the world has been searching for historical analog ##ies to help us understand what has been happening .",巴 黎 - 随 着 经 济 危 机 不 断 加 深 和 蔓 延 ， 整 个 世 界 一 直 在 寻 找 历 史 上 的 类 似 事 件 希 望 有 助 于 我 们 了 解 目 前 正 在 发 生 的 情 况 。,False
2,"At the start of the crisis , many people like ##ned it to 1982 or 1973 , which was reassuring , because both dates refer to classical c ##yclic ##al down ##turn ##s .",一 开 始 ， 很 多 人 把 这 次 危 机 比 作 1982 年 或 1973 年 所 发 生 的 情 况 ， 这 样 得 类 比 是 令 人 宽 心 的 ， 因 为 这 两 段 时 期 意 味 着 典 型 的 周 期 性 衰 退 。,False
3,The tendency is either excessive restraint ( Europe ) or a diffusion of the effort ( the United States ) .,目 前 的 趋 势 是 ， 要 么 是 过 度 的 克 制 （ 欧 洲 ） ， 要 么 是 努 力 的 扩 展 （ 美 国 ） 。,True
4,"Europe is being cautious in the name of avoiding debt and defending the euro , whereas the US has moved on many fronts in order not to waste an ideal opportunity to implement badly needed structural reforms .",欧 洲 在 避 免 债 务 和 捍 卫 欧 元 的 名 义 下 正 变 得 谨 慎 ， 而 美 国 已 经 在 许 多 方 面 行 动 起 来 ， 以 利 用 这 一 理 想 的 时 机 来 实 行 急 需 的 结 构 性 改 革 。,False
...,...,...,...
256087,He is part of the proud generation that defeated apartheid – and then peacefully engineered a transition to du ##rable black - majority rule .,他 是 打 败 种 族 隔 离 制 度 [UNK] [UNK] 然 后 让 南 非 和 平 过 渡 到 由 人 口 占 多 数 的 黑 人 进 行 长 期 统 治 [UNK] [UNK] 的 自 豪 的 一 代 人 的 一 部 分 。,False
256088,Their achievement remains one of the greatest in recent history .,这 一 代 人 的 成 就 是 现 代 史 上 最 伟 大 的 成 就 之 一 。,False
256089,"At the same time , Z ##uma ’ s revolutionary generation still seems uneasy leading South Africa in a post - apartheid era that is now 15 years old .",同 时 ， 祖 马 革 命 的 一 代 似 乎 对 领 导 打 破 种 族 隔 离 制 度 15 年 后 的 南 非 ， 还 不 适 应 。,False
256090,"In a region that re ##vere ##s the elderly , Z ##uma ’ s attachment to his rural traditions must be matched by an equal open ##ness to the appetite ##s of the country ’ s youth .",在 一 个 尊 敬 老 人 的 地 区 ， 祖 马 对 其 乡 下 传 统 的 依 恋 ， 必 须 和 对 南 非 年 青 人 的 爱 好 所 持 的 平 等 而 开 放 态 度 相 平 衡 。,False


In [ ]:
# skip
eda_tok_df = _get_eda_tok_df(tok_df)
eda_tok_df.describe()

,eng_len,chn_len
count,256092.000000,256092.000000
mean,24.718765,34.899095
std,10.524990,14.172736
min,1.000000,1.000000
25%,17.000000,24.000000
50%,24.000000,35.000000
75%,32.000000,46.000000
max,98.000000,62.000000


## get_datasets

In [ ]:
# export
def get_nc_dss(tok_data_loc, enc_tokenizer, dec_tokenizer, enc_seq_len, dec_seq_len, pct=1.0):
    tok_df = pd.read_csv(tok_data_loc)
    
    splits = ColSplitter()(tok_df)
    splits = pct_splits(splits, pct=pct)
    
    encoder_input_tfm = [attrgetter('English'), lambda x: x.split(' '), TransformersNumericalize(enc_tokenizer), Pad2Max(enc_seq_len, enc_tokenizer.pad_token_id)]
    decoder_input_tfm = [attrgetter('Chinese'), lambda x: x.split(' '), TransformersNumericalize(dec_tokenizer), Pad2Max(dec_seq_len+1, dec_tokenizer.pad_token_id), lambda x: x[:-1]]
    decoder_output_tfm = [attrgetter('Chinese'), lambda x: x.split(' '), TransformersNumericalize(dec_tokenizer), Pad2Max(dec_seq_len+1, dec_tokenizer.pad_token_id), lambda x: x[1:]]
    ds_tfms = [
        encoder_input_tfm,
        decoder_input_tfm,
        decoder_output_tfm,
    ]
    
    dss = Datasets(tok_df, tfms=ds_tfms, splits=splits, n_inp=2)
    return dss

In [ ]:
small_dss = get_nc_dss(tok_data_loc, enc_tokenizer, dec_tokenizer, enc_seq_len, dec_seq_len, pct=0.5)
dss = get_nc_dss(tok_data_loc, enc_tokenizer, dec_tokenizer, enc_seq_len, dec_seq_len)
test_eq(len(small_dss.train), len(dss.train)//2)

In [ ]:
dss.train[10], dss.decode(dss.train[10])

((TensorText([  101,  6355,   117,  1607,  1110, 17111,   117,  1105,  1103,  1646,
            117,  2693,  1157,  3407,  4812,  1111,  2052,   787,   188,  4265,
           5532,   117,  1336, 12982,  1107,  1618,  3571,  1190,  1211,  2182,
           1121,  1103,   182,  6533,  3954,   119,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
  TensorText([ 101, 5445, 1325, 1380, 3221,  679, 1062, 2398, 4638,  511, 2226, 5052,
          5401, 1744, 6206,  711, 2496,  791, 4638, 1059, 4413, 1314, 3322, 6566,
          3291, 1920, 4638

In [ ]:
dls = dss.dataloaders(bs=2)
for x in dls.train:
    print(x[0].shape, x[0].dtype, x[0].device, type(x[0]))
    print(x[1].shape, x[1].dtype, x[1].device, type(x[1]))
    print(x[2].shape, x[2].dtype, x[2].device, type(x[2]))
    break

torch.Size([2, 100]) torch.int64 cuda:0 <class 'fastai2.text.data.TensorText'>
torch.Size([2, 65]) torch.int64 cuda:0 <class 'fastai2.text.data.TensorText'>
torch.Size([2, 65]) torch.int64 cuda:0 <class 'fastai2.text.data.TensorText'>


## Export -

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 02_data.news_commentary.ipynb.
Converted 02_data.tatoeba.ipynb.
Converted 03a_models.patch.ipynb.
Converted 03c_models.bert2gpt2.ipynb.
Converted 03c_models.gru2gru.ipynb.
Converted 03c_models.tran2tran.ipynb.
Converted 04_metrics.ipynb.
Converted 99_fulltest_bert2gpt2.ipynb.
Converted 99_fulltest_gru2gru.ipynb.
Converted 99_fulltest_tran2tran.ipynb.
Converted index.ipynb.
